In [1]:
import geopy

In [2]:
geolocator = geopy.Nominatim()

In [3]:
location = geolocator.geocode('106 Corbett Avenue, SF')

In [4]:
location.latitude

37.7538681428571

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv('./Eviction_Notices.csv')

/usr/local/lib/python3.4/site-packages/pandas/io/parsers.py:1164: DtypeWarning: Columns (4,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [7]:
data['days'] = pd.to_datetime(data['File Date'])

In [8]:
data.days = data.days.map(lambda x: (pd.to_datetime('2015-05-15') - x).days)

In [9]:
def clean(x):
    try:
        split = x[1:-1].split(',')
        return float(split[0])
    except Exception:
        pass

In [10]:
data['lat'] = data.Client_Location.map(lambda x: clean(x))

In [11]:
def clean(x):
    try:
        split = x[1:-1].split(',')
        return float(split[1])
    except Exception:
        pass

In [12]:
data['lon'] = data.Client_Location.map(lambda x: clean(x))

In [13]:
data = data[data.lon < 0]

In [14]:
data = data[data.lat > 0]

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32623 entries, 0 to 33240
Data columns (total 33 columns):
Eviction_ID                32623 non-null object
Address                    32623 non-null object
City                       32622 non-null object
State                      32623 non-null object
Zip                        31837 non-null object
File Date                  32623 non-null object
Non Payment                32623 non-null bool
Breach                     32623 non-null bool
Nuisance                   32623 non-null bool
Illegal Use                32623 non-null bool
Failure to Sign Renewal    32623 non-null bool
Access Denial              32623 non-null bool
Unapproved Subtenant       32623 non-null bool
Owner Move In              32623 non-null bool
Demolition                 32623 non-null bool
Capital Improvement        32623 non-null bool
Substantial Rehab          32623 non-null bool
Ellis Act WithDrawal       32623 non-null bool
Condo Conversion           32623 

In [16]:
data.columns

Index(['Eviction_ID', 'Address', 'City', 'State', 'Zip', 'File Date', 'Non Payment', 'Breach', 'Nuisance', 'Illegal Use', 'Failure to Sign Renewal', 'Access Denial', 'Unapproved Subtenant', 'Owner Move In', 'Demolition', 'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal', 'Condo Conversion', 'Roommate Same Unit', 'Other Cause', 'Late Payments', 'Lead Remediation', 'Development', 'Good Samaritan Ends', 'Constraints', 'Constraints Date', 'Supervisor_District', 'Neighborhood', 'Client_Location', 'days', 'lat', 'lon'], dtype='object')

In [17]:
clean = data.copy()

In [18]:
reasons = ['Non Payment', 'Breach', 'Nuisance', 'Illegal Use', 'Failure to Sign Renewal', 'Access Denial', 'Unapproved Subtenant', 'Owner Move In', 'Demolition', 'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal', 'Condo Conversion', 'Roommate Same Unit', 'Other Cause', 'Late Payments', 'Lead Remediation', 'Development']

In [19]:
x = data[reasons].stack()

In [20]:
data['reason'] = pd.Series(pd.Categorical(x[x!=0].index.get_level_values(1)))

In [21]:
df = data[['lat', 'lon', 'days', 'reason']]

In [22]:
df.head()

,lat,lon,days,reason
0,37.794134,-122.417956,45,Nuisance
1,37.764190,-122.420438,45,Non Payment
2,37.775926,-122.424579,45,Access Denial
3,37.773209,-122.402243,45,Other Cause
4,37.782804,-122.454488,45,Nuisance


In [27]:
df.reason.astype('object', inplace=True)

0                    Nuisance
1                 Non Payment
2               Access Denial
3                 Other Cause
4                    Nuisance
5                      Breach
6                      Breach
7                      Breach
8               Access Denial
9               Owner Move In
10                     Breach
11                 Demolition
12                   Nuisance
13                   Nuisance
14                     Breach
15       Unapproved Subtenant
16              Owner Move In
17                     Breach
18              Owner Move In
19                     Breach
20                Other Cause
21                     Breach
22                     Breach
23              Owner Move In
24                     Breach
25                     Breach
26                Non Payment
27        Capital Improvement
28                     Breach
29        Capital Improvement
                 ...         
33209           Owner Move In
33210           Owner Move In
33211     

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32623 entries, 0 to 33240
Data columns (total 4 columns):
lat       32623 non-null float64
lon       32623 non-null float64
days      32623 non-null int64
reason    32623 non-null object
dtypes: float64(2), int64(1), object(1)
memory usage: 1.2+ MB


In [29]:
from sklearn.preprocessing import MinMaxScaler, Normalizer

scaler = MinMaxScaler()
n = Normalizer()

In [42]:
df[['lon', 'lat', 'days']] = scaler.fit_transform(df[['lon', 'lat', 'days']]) -.5

/usr/local/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.4/site-packages/pandas/core/indexing.py:411: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [43]:
df.head()

,lat,lon,days,reason
0,0.196697,0.131036,-0.5,Nuisance
1,-0.045288,0.114216,-0.5,Non Payment
2,0.049554,0.086153,-0.5,Access Denial
3,0.027600,0.237513,-0.5,Other Cause
4,0.105133,-0.116522,-0.5,Nuisance


In [48]:
df.info()

df = df[['days', 'lat', 'lon', 'reason']]
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32623 entries, 0 to 33240
Data columns (total 4 columns):
days      32623 non-null float64
lat       32623 non-null float64
lon       32623 non-null float64
reason    32623 non-null object
dtypes: float64(3), object(1)
memory usage: 1.2+ MB


,days,lat,lon,reason
0,-0.5,0.196697,0.131036,Nuisance
1,-0.5,-0.045288,0.114216,Non Payment
2,-0.5,0.049554,0.086153,Access Denial
3,-0.5,0.027600,0.237513,Other Cause
4,-0.5,0.105133,-0.116522,Nuisance


In [49]:
df.to_json('evictions.json')

In [46]:
df.days.describe()

count    32623.000000
mean         0.050855
std          0.308519
min         -0.500000
25%         -0.228760
50%          0.094866
75%          0.329180
max          0.500000
Name: days, dtype: float64

In [41]:
data['File Date'][-1:]


33240    01/02/1997
Name: File Date, dtype: object